### Dependencies and function definitions and grabbing model files

In [ ]:
from astropy.io import ascii
import numpy as np
import glob
import os
import sys
import progressbar

def import_data_files(files):
    alldata = []

    bar = progressbar.ProgressBar(maxval=len(files)).start()
    for i,file in enumerate(files):
        bar.update(i)
        if "ACS" in file:
            instrument = "ACS_WFC"
        if "WFC3" in file:
            instrument = "WFC3"
        if "WPC2" in file:
            instrument = "WPC2"
            
        basefile = os.path.basename(file)
        basefile = basefile.replace("M.track.eep.cmd", "")
        mass = int(basefile)/10000.
        table = ascii.read(file, header_start=14)
        alldata.append({'table':table, 'instrument':instrument, 'mass':mass})
        
    bar.finish()
    
    return(alldata)

def get_mass_limits(models, observations):
    
    masses = []
    limits = []
    for mass in sorted(np.unique([m['mass'] for m in models])):
        if mass>80:continue
        tables = [m['table'] for m in models if m['mass']==mass]
        absmag = {}
        for key in observations['photometry'].keys():
            dm = 5*np.log10(observations['dist'])+25.0
            absmag[key] = observations['photometry'][key] - dm - observations['extinction'][key]
        
        results = {}
        for key in absmag.keys():
            for table in tables:
                mask = table['phase'] == 0.0
                subtable = table[mask]
                if key in subtable.keys():
                    n = len(subtable[key]>absmag[key])
                    results[key]=n
                    break
                    
        if any([results[key]==0 for key in results.keys()]):
            masses.append(mass)
            limits.append(True)
        else:
            masses.append(mass)
            limits.append(False)
    
    return(np.array(masses), np.array(limits))

### Import all of the model files and output as a single object

In [ ]:
basepath='/Users/ckilpatrick/Dropbox/scripts/python/progenitors/sed/data/mist'
if os.path.exists(basepath):
    globstr = os.path.join(basepath, 'FEH_0000','*','*.cmd')
else:
    globstr = os.path.join('FEH_0000','*','*.cmd')

files = glob.glob(globstr)
nfiles = len(files)
print(f'There are {nfiles} files to grab')

allfiles = import_data_files(files)

In [ ]:
SN80D = {'photometry':{'ACS_WFC_F555W':26.8269, 'ACS_WFC_F814W':26.8553}, 
         'dist':22.583, 
         'extinction':{'ACS_WFC_F555W':0.044, 'ACS_WFC_F814W':0.024}}

masses, limits = get_mass_limits(allfiles, SN80D)
print(masses, limits)

In [ ]:
import numpy as np

# SN2020ZBV
f814w = 26.5717 
f555w = 25.9415

distance = 18.28
dm = 5*np.log10(distance)+25.0

a_555w = 0.0
a_814w = 0.0

In [ ]:
for absolute_magnitude,key in zip([f814w-dm,f555w-dm],['ACS_WFC_F814W','ACS_WFC_F555W']):
        
        print(absolute_magnitude)
        mask = cmd_table[key]>absolute_magnitude
        n = len(cmd_table[mask])
        
        print(n)

In [ ]:
import pandas as pd

df = pd.read_csv("Limited Mags.csv")
#df = df[df['Filter']=='acs.f55w']

print(df)